# Libraries


In [1]:
import git
from pyspark.sql.functions import*
import subprocess
import os
import sys
import time
from pyspark.sql import SparkSession

# Variables Globales

In [2]:
path_to_repo="velib_dlk/datalake/"
hdfs_repo_name="datalake"
hive_db_name="myVelib" # Ne pas oublier de créer la db via le terminal
table_name="my_velib_table_partioned"

#Démarrage d'une cession spark
spark = SparkSession.builder.appName("velib").enableHiveSupport().getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

23/05/04 11:03:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
#! cd /velib_dlk/datalake && git pull

# Functions

In [4]:
def pull_data(path_to_repo):
    result_git=subprocess.run(f"cd /{path_to_repo} && git pull", shell=True, check=True, capture_output=True, text=True).stdout[:-1]
    return result_git

In [5]:
def put_data_on_hdfs():
    temps_deb=time.time()
    !hadoop fs -put -p /velib_dlk/datalake /
    temps_fin=time.time()
    return temps_fin-temps_deb

In [6]:
def check_hive_table_is_empty(table_name):
    return spark.sql(f"SHOW TABLES LIKE '{table_name}'").count()       

In [7]:
def get_appended_file_names():

    # Open the Git repository
    repo = git.Repo(path_to_repo)

    # Get the most recent commit
    commit = repo.head.commit

    # Get the list of changed files in the commit
    changed_files = commit.stats.files.keys()
    
    l=len(changed_files)
    list_changed_files=[f"/{hdfs_repo_name}/{list(changed_files)[i]}" for i in range(l)]
    return list_changed_files


In [8]:
def run_cmd(args_list):
    """
    run linux commands
    """
    # import subprocess
    print('Running system command: {0}'.format(' '.join(args_list)))
    proc = subprocess.Popen(args_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    s_output, s_err = proc. communicate()
    s_return = proc.returncode
    return s_return, s_output, s_err

In [9]:
def get_hadoop_file_paths():
    table_list=[]
    (_,out_table, _)= run_cmd(['hdfs', 'dfs', '-ls', '/'+hdfs_repo_name ])
    for table_name in out_table.decode().split('\n'):
        if '.csv' in table_name: 
            table_list.append('/'+ table_name.split(' /')[1])
    return table_list

In [10]:
def main():
    spark.sql(f"USE {hive_db_name}")
    # pull data from repo
    etat_repo=pull_data(path_to_repo)
    # put data on HDFS Datalake
    time_pull_data= put_data_on_hdfs()
    
    print (etat_repo)
    print(f"temps de chargement hdfs:{time_pull_data} secondes")
  
    
    if check_hive_table_is_empty(table_name)==0:
        print("cas 1")

        df_velib = spark.read.csv('/datalake/*.csv', header=True, inferSchema=True)
    
        # Add columns for year, month, day, and hour to the DataFrame
        df_with_partitions = df_velib.withColumn("year", year(col("record_timestamp"))) \
                          .withColumn("month", month(col("record_timestamp"))) \
                          .withColumn("day", dayofmonth(col("record_timestamp"))) \
                          .withColumn("dayofweek", dayofweek(col("record_timestamp"))) \
                          .withColumn("hour", hour(col("record_timestamp")))

        # Partition the DataFrame by year, month, day, and hour
        df_with_partitions.write.mode("overwrite").partitionBy("year", "month", "day", "hour").format("parquet").saveAsTable(table_name)  
    
    elif etat_repo=="Already up to date.":
        print("up to date")
        
    else:
        print("cas update")
        file_paths=get_appended_file_names()
        print(file_paths)
        df_velib = spark.read.csv(file_paths, header=True, inferSchema=True)
        
        # Add columns for year, month, day, and hour to the DataFrame
        df_with_partitions = df_velib.withColumn("year", year(col("record_timestamp"))) \
                          .withColumn("month", month(col("record_timestamp"))) \
                          .withColumn("day", dayofmonth(col("record_timestamp"))) \
                          .withColumn("dayofweek", dayofweek(col("record_timestamp"))) \
                          .withColumn("hour", hour(col("record_timestamp")))
        
        
        df_with_partitions.write.partitionBy("year", "month", "day", "hour").mode("overwrite").format("parquet").option("partitionOverwriteMode", "dynamic").saveAsTable(table_name) 
    
    nb_lign_hive=spark.sql(f"SELECT count(*) from {table_name} ")
    count_int = int(nb_lign_hive.first()[0])
       
    print(nb_lign_hive) 
    return count_int

# Put data on Hive Datawarehouse

In [ ]:
nb_lign_hive= main()

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used


put: `/datalake/27042023_22_33_39.csv': File exists
put: `/datalake/12042023_12_49_38.csv': File exists
put: `/datalake/15042023_17_40_19.csv': File exists
put: `/datalake/10042023_14_29_38.csv': File exists
put: `/datalake/02052023_14_34_38.csv': File exists
put: `/datalake/16042023_04_02_49.csv': File exists
put: `/datalake/06042023_17_42_38.csv': File exists
put: `/datalake/20042023_08_47_38.csv': File exists
put: `/datalake/02052023_15_34_39.csv': File exists
put: `/datalake/25042023_12_25_38.csv': File exists
put: `/datalake/06042023_15_42_38.csv': File exists
put: `/datalake/26042023_16_09_38.csv': File exists
put: `/datalake/11042023_18_20_38.csv': File exists
put: `/datalake/06042023_12_42_37.csv': File exists
put: `/datalake/05042023_17_16_38.csv': File exists
put: `/datalake/06042023_07_26_37.csv': File exists
put: `/datalake/13042023_02_06_13.csv': File exists
put: `/datalake/16042023_03_02_09.csv': File exists
put: `/datalake/30042023_19_08_39.csv': File exists
put: `/datal

23/05/04 11:03:43 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/05/04 11:03:58 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/05/04 11:04:13 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/05/04 11:04:28 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/05/04 11:04:43 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/05/04 11:04:58 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registere

In [ ]:
df_velib_hdfs = spark.read.csv('/datalake/*.csv', header=True, inferSchema=True)
nb_lign_hdfs= df_velib_hdfs.count()
print(f"nb ligne hdfs:{nb_lign_hdfs}")

In [ ]:
#hive 17146240
#hdfs 17204640
#-63320
res=nb_lign_hive-nb_lign_hdfs
print(f"différence = {res}")

In [ ]:
#df_velib_local = spark.read.csv('file:///velib_dlk/datalake/*.csv', header=True, inferSchema=True)
#df_velib_local.count()